In [1]:
import sys
sys.path.append('/Users/remirosenthal/workshop')

In [2]:
import pandas as pd

from workshop.io import get_data
from workshop.classes import init

# Strategy 2

Respect min constraint (here accounting cost is not a problem)

- compute groups/days matrix : groups as rows and days as columns (5000 x 100) fill NaN by -1
- sort days (columns) from least demanded to most demanded
- for each column :
  -> pick 125 (min constraint) lowest cost
  -> update matrix

At this stage, min constraint is satisfied. We can now place groups in best fitting day fashion.

- sort groups by "annoyance" criterion (-> bigger groups that want most wanted days are diffcult to assign)
- for each group :
  -> score its desired days
  -> assign it to the min cost
  -> update assignments
  -> if no desired days is still available, score every day and assign it to the min

In [3]:
data = get_data()
maker = init(data)

### compute order of days

In [4]:
%time
demand = pd.Series(0, index=range(1, 101))
for group in maker.groups:
    demand.loc[group.days] += group.size
demand = demand.sort_values(ascending=True)
days_order = demand.index.tolist()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.05 µs


### compute matrix and sort

In [5]:
%time
matrix = pd.DataFrame()

for group in maker.groups:
    for day in group.days:
        matrix.loc[group, day] = group.evaluate(day)
    
matrix = matrix[days_order]

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.05 µs


### fill min constraint

In [6]:
%time
for day in matrix.columns:
    scores = matrix[day].dropna().sort_values(ascending=True)
    count = 0
    for group, _ in scores.iteritems():
        maker.assign(group, day)
        matrix = matrix.drop(group)
        count += group.size
        if count >= maker.MIN:
            break
    
    # if not enough people are in, force some to be there
    # # -> to do more flexible ones ?
    
    if count < maker.MIN:
        print("oops")
        group_scored = [(group, group.score(day), size) for group in maker.groups]
        group_scored = sorted(group_scored, key=lambda x: x[1])
        for group in group_scored:
            maker.assign(group, day)
            matrix.drop(group)
            count += group.size
            if count >= maker.MIN:
                break

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs
**assigned group number 3579 to day 93.
current bookings 2
**assigned group number 3539 to day 93.
current bookings 6
**assigned group number 3382 to day 93.
current bookings 10
**assigned group number 3300 to day 93.
current bookings 12
**assigned group number 3226 to day 93.
current bookings 15
**assigned group number 3086 to day 93.
current bookings 19
**assigned group number 352 to day 93.
current bookings 23
**assigned group number 469 to day 93.
current bookings 29
**assigned group number 1909 to day 93.
current bookings 32
**assigned group number 4580 to day 93.
current bookings 34
**assigned group number 256 to day 93.
current bookings 38
**assigned group number 2304 to day 93.
current bookings 43
**assigned group number 2654 to day 93.
current bookings 48
**assigned group number 1379 to day 93.
current bookings 52
**assigned group number 1994 to day 93.
current bookings 55
**assigned group number 2980 to day 93.
c

**assigned group number 3729 to day 84.
current bookings 620
**assigned group number 1053 to day 84.
current bookings 624
**assigned group number 4352 to day 84.
current bookings 628
**assigned group number 776 to day 84.
current bookings 632
**assigned group number 1702 to day 99.
current bookings 638
**assigned group number 622 to day 99.
current bookings 642
**assigned group number 775 to day 99.
current bookings 644
**assigned group number 3913 to day 99.
current bookings 646
**assigned group number 2988 to day 99.
current bookings 648
**assigned group number 2130 to day 99.
current bookings 652
**assigned group number 889 to day 99.
current bookings 655
**assigned group number 364 to day 99.
current bookings 658
**assigned group number 3204 to day 99.
current bookings 660
**assigned group number 4016 to day 99.
current bookings 662
**assigned group number 2240 to day 99.
current bookings 666
**assigned group number 1540 to day 99.
current bookings 669
**assigned group number 4578 

**assigned group number 2415 to day 72.
current bookings 1359
**assigned group number 3599 to day 72.
current bookings 1361
**assigned group number 396 to day 72.
current bookings 1364
**assigned group number 369 to day 72.
current bookings 1367
**assigned group number 2274 to day 72.
current bookings 1371
**assigned group number 210 to day 72.
current bookings 1375
**assigned group number 4980 to day 72.
current bookings 1379
**assigned group number 3423 to day 72.
current bookings 1383
**assigned group number 4173 to day 72.
current bookings 1387
**assigned group number 946 to day 72.
current bookings 1392
**assigned group number 3824 to day 72.
current bookings 1397
**assigned group number 2585 to day 98.
current bookings 1404
**assigned group number 3433 to day 98.
current bookings 1407
**assigned group number 2030 to day 98.
current bookings 1410
**assigned group number 3368 to day 98.
current bookings 1414
**assigned group number 2999 to day 98.
current bookings 1419
**assigned g

**assigned group number 659 to day 69.
current bookings 2069
**assigned group number 4422 to day 69.
current bookings 2071
**assigned group number 4131 to day 69.
current bookings 2076
**assigned group number 1937 to day 69.
current bookings 2080
**assigned group number 118 to day 69.
current bookings 2084
**assigned group number 54 to day 69.
current bookings 2087
**assigned group number 421 to day 69.
current bookings 2090
**assigned group number 844 to day 69.
current bookings 2094
**assigned group number 4473 to day 69.
current bookings 2099
**assigned group number 3857 to day 69.
current bookings 2103
**assigned group number 1951 to day 69.
current bookings 2106
**assigned group number 3710 to day 69.
current bookings 2110
**assigned group number 4674 to day 69.
current bookings 2113
**assigned group number 3496 to day 69.
current bookings 2116
**assigned group number 4207 to day 69.
current bookings 2119
**assigned group number 276 to day 69.
current bookings 2123
**assigned grou

**assigned group number 2519 to day 91.
current bookings 2753
**assigned group number 1365 to day 91.
current bookings 2755
**assigned group number 1192 to day 91.
current bookings 2763
**assigned group number 1546 to day 91.
current bookings 2770
**assigned group number 3384 to day 91.
current bookings 2775
**assigned group number 4215 to day 91.
current bookings 2780
**assigned group number 771 to day 91.
current bookings 2785
**assigned group number 1987 to day 91.
current bookings 2792
**assigned group number 3096 to day 76.
current bookings 2799
**assigned group number 132 to day 76.
current bookings 2801
**assigned group number 3545 to day 76.
current bookings 2805
**assigned group number 2184 to day 76.
current bookings 2809
**assigned group number 3641 to day 76.
current bookings 2812
**assigned group number 4166 to day 76.
current bookings 2814
**assigned group number 674 to day 76.
current bookings 2818
**assigned group number 3350 to day 76.
current bookings 2821
**assigned 

...

**assigned group number 2932 to day 54.
current bookings 9535
**assigned group number 2974 to day 54.
current bookings 9538
**assigned group number 816 to day 54.
current bookings 9542
**assigned group number 1684 to day 54.
current bookings 9544
**assigned group number 3129 to day 54.
current bookings 9548
**assigned group number 3193 to day 54.
current bookings 9551
**assigned group number 751 to day 54.
current bookings 9559
**assigned group number 3197 to day 54.
current bookings 9564
**assigned group number 660 to day 54.
current bookings 9572
**assigned group number 3302 to day 54.
current bookings 9576
**assigned group number 3456 to day 54.
current bookings 9579
**assigned group number 3559 to day 54.
current bookings 9583
**assigned group number 3600 to day 54.
current bookings 9587
**assigned group number 3713 to day 54.
current bookings 9589
**assigned group number 3747 to day 54.
current bookings 9593
**assigned group number 476 to day 54.
current bookings 9596
**assigned g

**assigned group number 4545 to day 46.
current bookings 10210
**assigned group number 1010 to day 46.
current bookings 10214
**assigned group number 1002 to day 46.
current bookings 10217
**assigned group number 993 to day 46.
current bookings 10222
**assigned group number 4644 to day 46.
current bookings 10226
**assigned group number 953 to day 46.
current bookings 10229
**assigned group number 938 to day 46.
current bookings 10235
**assigned group number 777 to day 46.
current bookings 10237
**assigned group number 748 to day 46.
current bookings 10241
**assigned group number 708 to day 46.
current bookings 10248
**assigned group number 685 to day 46.
current bookings 10251
**assigned group number 1627 to day 46.
current bookings 10254
**assigned group number 1729 to day 46.
current bookings 10257
**assigned group number 4348 to day 46.
current bookings 10261
**assigned group number 1774 to day 46.
current bookings 10266
**assigned group number 3181 to day 46.
current bookings 10271

**assigned group number 1547 to day 47.
current bookings 10927
**assigned group number 4 to day 53.
current bookings 10931
**assigned group number 2541 to day 53.
current bookings 10937
**assigned group number 2552 to day 53.
current bookings 10941
**assigned group number 2615 to day 53.
current bookings 10945
**assigned group number 961 to day 53.
current bookings 10951
**assigned group number 880 to day 53.
current bookings 10956
**assigned group number 4069 to day 53.
current bookings 10961
**assigned group number 817 to day 53.
current bookings 10965
**assigned group number 813 to day 53.
current bookings 10969
**assigned group number 2528 to day 53.
current bookings 10975
**assigned group number 2748 to day 53.
current bookings 10978
**assigned group number 2850 to day 53.
current bookings 10983
**assigned group number 4633 to day 53.
current bookings 10985
**assigned group number 3832 to day 53.
current bookings 10988
**assigned group number 2877 to day 53.
current bookings 10992

**assigned group number 1201 to day 11.
current bookings 11612
**assigned group number 1182 to day 11.
current bookings 11615
**assigned group number 1180 to day 11.
current bookings 11619
**assigned group number 4291 to day 11.
current bookings 11621
**assigned group number 4331 to day 11.
current bookings 11626
**assigned group number 4334 to day 11.
current bookings 11631
**assigned group number 1114 to day 11.
current bookings 11635
**assigned group number 4419 to day 11.
current bookings 11639
**assigned group number 3725 to day 11.
current bookings 11644
**assigned group number 4461 to day 11.
current bookings 11646
**assigned group number 1013 to day 11.
current bookings 11649
**assigned group number 988 to day 11.
current bookings 11651
**assigned group number 2846 to day 11.
current bookings 11655
**assigned group number 2853 to day 11.
current bookings 11658
**assigned group number 895 to day 11.
current bookings 11663
**assigned group number 872 to day 11.
current bookings 1

**assigned group number 2388 to day 26.
current bookings 12296
**assigned group number 2159 to day 26.
current bookings 12301
**assigned group number 2132 to day 26.
current bookings 12305
**assigned group number 2410 to day 26.
current bookings 12308
**assigned group number 2469 to day 26.
current bookings 12316
**assigned group number 2097 to day 26.
current bookings 12323
**assigned group number 1992 to day 5.
current bookings 12328
**assigned group number 2550 to day 5.
current bookings 12331
**assigned group number 4273 to day 5.
current bookings 12334
**assigned group number 873 to day 5.
current bookings 12342
**assigned group number 845 to day 5.
current bookings 12347
**assigned group number 791 to day 5.
current bookings 12350
**assigned group number 4294 to day 5.
current bookings 12353
**assigned group number 4301 to day 5.
current bookings 12358
**assigned group number 4309 to day 5.
current bookings 12363
**assigned group number 3730 to day 5.
current bookings 12369
**ass

### sort remaning groups and assign

In [7]:
%time
# todo : is there a better way of sorting groups ?
for group in sorted(maker.groups, key=lambda x: x.size, reverse=True):
    day = maker.get_best_wish(group)
    maker.assign(group, day)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs
**assigned group number 41 to day 31.
current bookings 12713
**assigned group number 42 to day 81.
current bookings 12721
**assigned group number 66 to day 1.
current bookings 12729
**assigned group number 127 to day 2.
current bookings 12737
**assigned group number 134 to day 18.
current bookings 12745
**assigned group number 250 to day 1.
current bookings 12753
**assigned group number 295 to day 10.
current bookings 12761
**assigned group number 350 to day 96.
current bookings 12769
**assigned group number 428 to day 40.
current bookings 12777
**assigned group number 486 to day 11.
current bookings 12785
**assigned group number 504 to day 1.
current bookings 12793
**assigned group number 824 to day 60.
current bookings 12801
**assigned group number 1081 to day 25.
current bookings 12809
**assigned group number 1110 to day 1.
current bookings 12817
**assigned group number 1176 to day 54.
current bookings 12825
**assigned 

**assigned group number 4126 to day 11.
current bookings 13786
**assigned group number 4137 to day 68.
current bookings 13793
**assigned group number 4141 to day 45.
current bookings 13800
**assigned group number 4226 to day 88.
current bookings 13807
**assigned group number 4240 to day 17.
current bookings 13814
**assigned group number 4249 to day 87.
current bookings 13821
**assigned group number 4254 to day 31.
current bookings 13828
**assigned group number 4278 to day 31.
current bookings 13835
**assigned group number 4290 to day 13.
current bookings 13842
**assigned group number 4300 to day 87.
current bookings 13849
**assigned group number 4303 to day 81.
current bookings 13856
**assigned group number 4321 to day 39.
current bookings 13863
**assigned group number 4368 to day 66.
current bookings 13870
**assigned group number 4468 to day 88.
current bookings 13877
**assigned group number 4471 to day 75.
current bookings 13884
**assigned group number 4549 to day 45.
current booking

**assigned group number 3054 to day 4.
current bookings 14647
**assigned group number 3059 to day 24.
current bookings 14653
**assigned group number 3079 to day 4.
current bookings 14659
**assigned group number 3095 to day 46.
current bookings 14665
**assigned group number 3136 to day 33.
current bookings 14671
**assigned group number 3138 to day 23.
current bookings 14677
**assigned group number 3168 to day 59.
current bookings 14683
**assigned group number 3169 to day 61.
current bookings 14689
**assigned group number 3174 to day 39.
current bookings 14695
**assigned group number 3228 to day 66.
current bookings 14701
**assigned group number 3244 to day 66.
current bookings 14707
**assigned group number 3280 to day 88.
current bookings 14713
**assigned group number 3343 to day 15.
current bookings 14719
**assigned group number 3353 to day 94.
current bookings 14725
**assigned group number 3356 to day 26.
current bookings 14731
**assigned group number 3365 to day 19.
current bookings 

**assigned group number 415 to day 24.
current bookings 15404
**assigned group number 419 to day 26.
current bookings 15409
**assigned group number 453 to day 51.
current bookings 15414
**assigned group number 456 to day 94.
current bookings 15419
**assigned group number 463 to day 18.
current bookings 15424
**assigned group number 464 to day 46.
current bookings 15429
**assigned group number 475 to day 33.
current bookings 15434
**assigned group number 529 to day 25.
current bookings 15439
**assigned group number 567 to day 76.
current bookings 15444
**assigned group number 574 to day 88.
current bookings 15449
**assigned group number 590 to day 90.
current bookings 15454
**assigned group number 593 to day 23.
current bookings 15459
**assigned group number 625 to day 1.
current bookings 15464
**assigned group number 651 to day 46.
current bookings 15469
**assigned group number 679 to day 23.
current bookings 15474
**assigned group number 713 to day 25.
current bookings 15479
**assigne

**assigned group number 2975 to day 53.
current bookings 16104
**assigned group number 2997 to day 34.
current bookings 16109
**assigned group number 3005 to day 94.
current bookings 16114
**assigned group number 3044 to day 47.
current bookings 16119
**assigned group number 3046 to day 23.
current bookings 16124
**assigned group number 3074 to day 80.
current bookings 16129
**assigned group number 3091 to day 18.
current bookings 16134
**assigned group number 3104 to day 29.
current bookings 16139
**assigned group number 3144 to day 45.
current bookings 16144
**assigned group number 3156 to day 74.
current bookings 16149
**assigned group number 3198 to day 46.
current bookings 16154
**assigned group number 3205 to day 41.
current bookings 16159
**assigned group number 3222 to day 89.
current bookings 16164
**assigned group number 3224 to day 51.
current bookings 16169
**assigned group number 3229 to day 38.
current bookings 16174
**assigned group number 3237 to day 2.
current bookings

...

**assigned group number 349 to day 36.
current bookings 19486
**assigned group number 358 to day 11.
current bookings 19489
**assigned group number 362 to day 81.
current bookings 19492
**assigned group number 378 to day 25.
current bookings 19495
**assigned group number 405 to day 54.
current bookings 19498
**assigned group number 417 to day 74.
current bookings 19501
**assigned group number 433 to day 1.
current bookings 19504
**assigned group number 438 to day 25.
current bookings 19507
**assigned group number 449 to day 18.
current bookings 19510
**assigned group number 451 to day 50.
current bookings 19513
**assigned group number 454 to day 17.
current bookings 19516
**assigned group number 492 to day 8.
current bookings 19519
**assigned group number 514 to day 3.
current bookings 19522
**assigned group number 531 to day 2.
current bookings 19525
**assigned group number 549 to day 37.
current bookings 19528
**assigned group number 570 to day 16.
current bookings 19531
**assigned g

**assigned group number 2595 to day 22.
current bookings 19885
**assigned group number 2617 to day 36.
current bookings 19888
**assigned group number 2622 to day 21.
current bookings 19891
**assigned group number 2644 to day 86.
current bookings 19894
**assigned group number 2648 to day 10.
current bookings 19897
**assigned group number 2703 to day 44.
current bookings 19900
**assigned group number 2791 to day 26.
current bookings 19903
**assigned group number 2803 to day 37.
current bookings 19906
**assigned group number 2822 to day 88.
current bookings 19909
**assigned group number 2832 to day 81.
current bookings 19912
**assigned group number 2845 to day 10.
current bookings 19915
**assigned group number 2854 to day 13.
current bookings 19918
**assigned group number 2865 to day 68.
current bookings 19921
**assigned group number 2873 to day 90.
current bookings 19924
**assigned group number 2890 to day 2.
current bookings 19927
**assigned group number 2891 to day 20.
current bookings

**assigned group number 4424 to day 13.
current bookings 20302
**assigned group number 4436 to day 74.
current bookings 20305
**assigned group number 4441 to day 75.
current bookings 20308
**assigned group number 4449 to day 47.
current bookings 20311
**assigned group number 4460 to day 14.
current bookings 20314
**assigned group number 4479 to day 41.
current bookings 20317
**assigned group number 4540 to day 38.
current bookings 20320
**assigned group number 4553 to day 74.
current bookings 20323
**assigned group number 4569 to day 91.
current bookings 20326
**assigned group number 4587 to day 4.
current bookings 20329
**assigned group number 4592 to day 27.
current bookings 20332
**assigned group number 4604 to day 78.
current bookings 20335
**assigned group number 4620 to day 43.
current bookings 20338
**assigned group number 4625 to day 10.
current bookings 20341
**assigned group number 4634 to day 51.
current bookings 20344
**assigned group number 4673 to day 9.
current bookings 

**assigned group number 1825 to day 32.
current bookings 20627
**assigned group number 1866 to day 47.
current bookings 20629
**assigned group number 1920 to day 33.
current bookings 20631
**assigned group number 1957 to day 33.
current bookings 20633
**assigned group number 1976 to day 5.
current bookings 20635
**assigned group number 2033 to day 51.
current bookings 20637
**assigned group number 2055 to day 10.
current bookings 20639
**assigned group number 2061 to day 22.
current bookings 20641
**assigned group number 2066 to day 53.
current bookings 20643
**assigned group number 2070 to day 21.
current bookings 20645
**assigned group number 2086 to day 13.
current bookings 20647
**assigned group number 2094 to day 5.
current bookings 20649
**assigned group number 2138 to day 73.
current bookings 20651
**assigned group number 2160 to day 29.
current bookings 20653
**assigned group number 2161 to day 74.
current bookings 20655
**assigned group number 2201 to day 33.
current bookings 

**assigned group number 4197 to day 5.
current bookings 20891
**assigned group number 4203 to day 8.
current bookings 20893
**assigned group number 4214 to day 81.
current bookings 20895
**assigned group number 4218 to day 51.
current bookings 20897
**assigned group number 4288 to day 51.
current bookings 20899
**assigned group number 4292 to day 8.
current bookings 20901
**assigned group number 4295 to day 19.
current bookings 20903
**assigned group number 4316 to day 57.
current bookings 20905
**assigned group number 4330 to day 8.
current bookings 20907
**assigned group number 4339 to day 18.
current bookings 20909
**assigned group number 4356 to day 60.
current bookings 20911
**assigned group number 4372 to day 7.
current bookings 20913
**assigned group number 4379 to day 20.
current bookings 20915
**assigned group number 4387 to day 21.
current bookings 20917
**assigned group number 4411 to day 60.
current bookings 20919
**assigned group number 4433 to day 16.
current bookings 209

In [12]:
maker.make_submission("strat2")